Iteration 4

In [5]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [40]:
# Import dataset
dataFrame = spark.read.csv('Dataset/violenceData.csv',header=True,inferSchema=True)

In [41]:
# Show the dataset
dataFrame.show()

+-----------------+--------------------+----+------+-----+-----+-------+------+------+------+-------+
|CountiresandAreas|AnyViolentDiscipline|Male|Female|Urban|Rural|Poorest|Second|Middle|Fourth|Richest|
+-----------------+--------------------+----+------+-----+-----+-------+------+------+------+-------+
|      Afghanistan|                  74|  75|    74|   78|   74|     74|    74|    75|    74|     76|
|          Albania|                  77|  81|    73|   70|   82|     86|    80|    80|    74|     60|
|          Algeria|                  86|  88|    85|   87|   86|     86|    88|    87|    87|     84|
|          Andorra|                null|null|  null| null| null|   null|  null|  null|  null|   null|
|           Angola|                null|null|  null| null| null|   null|  null|  null|  null|   null|
|AntiguaandBarbuda|                null|null|  null| null| null|   null|  null|  null|  null|   null|
|        Argentina|                  72|  74|    71| null| null|     76|    74|   

In [10]:
# Show orginal columns before cleaning
print("Total data columns:", dataFrame.count())


Total data columns: 197


In [46]:
# drop empty rows
droppedTable = dataFrame.na.drop()
droppedTable.show()
droppedTable.count()




+--------------------+--------------------+----+------+-----+-----+-------+------+------+------+-------+
|   CountiresandAreas|AnyViolentDiscipline|Male|Female|Urban|Rural|Poorest|Second|Middle|Fourth|Richest|
+--------------------+--------------------+----+------+-----+-----+-------+------+------+------+-------+
|         Afghanistan|                  74|  75|    74|   78|   74|     74|    74|    75|    74|     76|
|             Albania|                  77|  81|    73|   70|   82|     86|    80|    80|    74|     60|
|             Algeria|                  86|  88|    85|   87|   86|     86|    88|    87|    87|     84|
|             Armenia|                  69|  71|    67|   68|   71|     73|    69|    69|    65|     68|
|          Azerbaijan|                  77|  80|    74|   76|   78|     81|    76|    76|    75|     74|
|          Bangladesh|                  82|  83|    82|   77|   84|     87|    85|    83|    81|     73|
|            Barbados|                  75|  78|    72|

64

In [ ]:
#